In [1]:
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
from qcodes import Instrument, ManualParameter

from quantify_core.measurement import MeasurementControl
from quantify_core.visualization.pyqt_plotmon import PlotMonitor_pyqt

from quantify_core.data.handling import (
    get_tuids_containing,
    set_datadir,
    default_datadir
)

rng = np.random.default_rng(seed=555555)
# Display any variable or statement on its own line
InteractiveShell.ast_node_interactivity = "all"

In [2]:
set_datadir("./quantify-data/")


In [3]:
class Device(Instrument):
    """A dummy instrument."""

    def __init__(self, name: str):
        super().__init__(name=name)

        self.add_parameter(name="amp_0", unit="A", parameter_class=ManualParameter)
        self.add_parameter(name="amp_1", unit="A", parameter_class=ManualParameter)
        self.add_parameter(name="offset", unit="A", parameter_class=ManualParameter)

        self.add_parameter(
            name="adc", label="ADC input", unit="V", get_cmd=self._get_dac_value
        )

    def _get_dac_value(self):
        s1 = np.exp(-3 * self.amp_0()) * np.sin(self.amp_0() * 2 * np.pi * 3)
        s2 = np.cos(self.amp_1() * 2 * np.pi * 2)
        return s1 + s2 + rng.uniform(0, 0.2) + self.offset()

In [4]:
meas_ctrl = MeasurementControl("meas_ctrl")
plotmon = PlotMonitor_pyqt("PlotMonitor")
meas_ctrl.instr_plotmon(plotmon.name)
device = Device("Device")


qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: eglfs, linuxfb, minimal, minimalegl, offscreen, vnc, wayland-egl, wayland, wayland-xcomposite-egl, wayland-xcomposite-glx, webgl, xcb.



ClosedError: 

In [ ]:
plotmon.tuids()
plotmon.tuids_extra()

In [ ]:
# set initial values to emulate the instrument state
device.amp_0(0.0)
device.amp_1(0.0)
device.offset(0.0)

n_pnts = 50

meas_ctrl.settables(device.amp_0)
meas_ctrl.setpoints(np.linspace(0, 1, n_pnts))
meas_ctrl.gettables(device.adc)
dset = meas_ctrl.run("ADC scan")


In [12]:
import h5py
import numpy as np
a = h5py.File("quantify-data/20231019/20231019-143422-469-8058a0-Frequency sweep/dataset.hdf5")
print(np.array(a["x0"]))

[5.0000000e+09 5.0020202e+09 5.0040404e+09 ... 5.1959596e+09 5.1979798e+09
 5.2000000e+09]


In [1]:
from qcodes.interactive_widget import experiments_widget

experiments_widget()

In [4]:
from quantify_core.data.experiment import QuantifyExperiment
s = QuantifyExperiment("20231019-143422-469-8058a0")



In [5]:
print(s)

<quantify_core.data.experiment.QuantifyExperiment at %x7fbfe5a15520>: TUID 20231019-143422-469-8058a0


In [6]:
d = s.load_dataset()

In [7]:
d

<xarray.Dataset>
Dimensions:  (dim_0: 10000)
Coordinates:
    x0       (dim_0) float64 5e+09 5.002e+09 5.004e+09 ... 5.198e+09 5.2e+09
    x1       (dim_0) float64 -3.0 -3.0 -3.0 -3.0 -3.0 ... 3.0 3.0 3.0 3.0 3.0
Dimensions without coordinates: dim_0
Data variables:
    y0       (dim_0) float64 -145.8 -146.6 -149.5 -146.7 ... 157.8 156.5 158.2
Attributes:
    tuid:                             20231019-143422-469-8058a0
    name:                             Frequency sweep
    grid_2d:                          True
    grid_2d_uniformly_spaced:         True
    1d_2_settables_uniformly_spaced:  False
    xlen:                             100
    ylen:                             100

In [8]:
experiments_widget(None, [d])

AttributeError: 'Dataset' object has no attribute 'run_id'